# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import numpy as np
from datasets import load_dataset

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

dataset = load_dataset("agentlans/grammar-classification")
dataset["train"][0]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'text': 'We have carried out a transaction cost analysis to help us understand the quality of execution on the execution venues where we execute/place orders.',
 'grammar': 1}

In [6]:
def tokenizing_map(dataset_entry):
    return tokenizer(dataset_entry["text"], padding="max_length", truncation=True)

# temp small datasets to quickly iterate on config
small_validation_dataset = dataset["validation"].select(range(100)).rename_column("grammar","labels")
small_train_dataset = dataset["train"].select(range(100)).rename_column("grammar","labels")

small_validation_dataset=small_validation_dataset.map(tokenizing_map, batched=True)
small_train_dataset=small_train_dataset.map(tokenizing_map, batched=True)


for param in model.base_model.parameters():
    param.requires_grad = False
    
small_train_dataset

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 100
})

In [7]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return { "accuracy": (predictions == labels).mean() }

trainer = Trainer(
    model=model,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    args=TrainingArguments(
        output_dir="./data/grammar",
        evaluation_strategy="epoch",
        num_train_epochs=3
    ),
    train_dataset=small_train_dataset,
    eval_dataset=small_validation_dataset,
    compute_metrics=compute_metrics
)
trainer.train()
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.706190,0.470000
2,No log,0.718949,0.470000
3,No log,0.723566,0.470000


{'eval_loss': 0.7235659956932068,
 'eval_accuracy': 0.47,
 'eval_runtime': 1.5818,
 'eval_samples_per_second': 63.22,
 'eval_steps_per_second': 8.219,
 'epoch': 3.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [8]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    target_modules = ["classifier"],
    modules_to_save = ["classifier"]
)

lora_model = get_peft_model(model, config)

In [9]:
from transformers import default_data_collator
lora_trainer = Trainer(
    model=lora_model,
    data_collator=default_data_collator,
    args=TrainingArguments(
        output_dir="./data/grammar",
        evaluation_strategy="epoch",
        num_train_epochs=3
    ),
    train_dataset=small_train_dataset,
    eval_dataset=small_validation_dataset,
    compute_metrics=compute_metrics
)
lora_model.print_trainable_parameters()
print ( lora_trainer.train_dataset[0])
print((small_train_dataset[0]))
print((small_validation_dataset[0]))
lora_trainer.train()
lora_trainer.evaluate()

trainable params: 1,196,580 || all params: 67,559,460 || trainable%: 1.7711509239416656
{'text': 'We have carried out a transaction cost analysis to help us understand the quality of execution on the execution venues where we execute/place orders.', 'labels': 1, 'input_ids': [101, 2057, 2031, 3344, 2041, 1037, 12598, 3465, 4106, 2000, 2393, 2149, 3305, 1996, 3737, 1997, 7781, 2006, 1996, 7781, 9356, 2073, 2057, 15389, 1013, 2173, 4449, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

IndexError: Invalid key: 98 is out of bounds for size 0

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [ ]:
# Saving the model
model.save("/tmp/your_model_name")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.